In [1]:
import os
import pandas as pd

pd.options.plotting.backend = 'plotly'
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn import metrics
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, \
                             GradientBoostingRegressor

from Pélec_04_fonctions import reg_modelGrid, visuRMSEGrid


In [2]:
write_data = True

if write_data is True:
    try:
        os.mkdir("./Figures/")
    except OSError as error:
        print(error)
    try:
        os.mkdir("./Tableaux/")
    except OSError as error:
        print(error)
else:
    print("""Visualisation uniquement dans le notebook
    pas de création de figures ni de tableaux""")


[Errno 17] File exists: './Figures/'
[Errno 17] File exists: './Tableaux/'


In [3]:
BEB = pd.read_csv('BEB.csv')

BEBM = BEB.drop(columns=['SiteEnergyUse(kBtu)', 'TotalGHGEmissions'])
SiteEnergyUse = np.array(BEB['SiteEnergyUse(kBtu)']).reshape(-1, 1)
TotalGHGEmissions = np.array(BEB.TotalGHGEmissions).reshape(-1, 1)

BEBM_train, BEBM_test, TotalGHGEmissions_train, TotalGHGEmissions_test = train_test_split(
    BEBM, TotalGHGEmissions, test_size=.2)

score = 'neg_root_mean_squared_error'


In [4]:
# Scaler moins sensible aux outlier d'après la doc
scaler = RobustScaler(quantile_range=(10, 90))


 # 1. Modèle de prédiction sur les émissions (TotalGHGEmissions)
 ## 1.1 Émissions brutes

 ### 1.1.1 Modèle LinearRegression

In [36]:
# modèle régression linéaire
pipeLR = make_pipeline(scaler, LinearRegression())

pipeLR.fit(BEBM_train, TotalGHGEmissions_train)

TotalGHGEmissions_predLR = pipeLR.predict(BEBM_test)

LRr2 = metrics.r2_score(TotalGHGEmissions_test, TotalGHGEmissions_predLR)
print("r2 :", LRr2)
LRrmse = metrics.mean_squared_error(TotalGHGEmissions_test,
                                    TotalGHGEmissions_predLR,
                                    squared=False)
print("rmse :", LRrmse)

fig = px.scatter(
    x=TotalGHGEmissions_predLR.squeeze(),
    y=TotalGHGEmissions_test.squeeze(),
    labels={
        'x': f'{TotalGHGEmissions_predLR=}'.partition('=')[0],
        'y': f'{TotalGHGEmissions_test=}'.partition('=')[0]
    },
    title=
    "Visualisation des données d'émissions prédites par le modèle de régression linéaire<br>vs les données test"
)
fig.show()


r2 : -0.3095599261157267
rmse : 583.6520545460625


 ### 1.1.2 Modèle Ridge

In [6]:
# régression ridge
# réglage des paramètre pour la gridsearch
alphasridge = np.logspace(-3, 5, 1000)
param_gridRidge = {'ridge__alpha': alphasridge}

GridRidge, \
BestParametresRidge, \
ScoresRidge, \
TotalGHGEmissions_predRidge, \
figRidge = reg_modelGrid(model=Ridge(),
                            scaler=scaler,
                            X_train=BEBM_train,
                            X_test=BEBM_test,
                            y_train=TotalGHGEmissions_train,
                            y_test=TotalGHGEmissions_test,
                            y_test_name='TotalGHGEmissions_test',
                            y_pred_name='TotalGHGEmissions_predRidge',
                            score=score,
                            param_grid=param_gridRidge)

print(BestParametresRidge)
print(ScoresRidge)
figRidge.show()

      paramètre    Ridge()
0  ridge__alpha  44.120929
         Ridge()
R²     -0.228390
RMSE  565.274608
MAE   121.669174


In [7]:
# graph visualisation RMSE Ridge pour les paramètres de GridSearchCV
FigRMSEGRidRidge = visuRMSEGrid(Ridge(), 'Ridge', alphasridge, 'alpha',
                                GridRidge)
FigRMSEGRidRidge.show()
if write_data is True:
    FigRMSEGRidRidge.write_image('./Figures/EmissionsGraphRMSERidge.pdf')


 ### 1.1.3 Modèle Lasso

In [8]:
# régression lasso
# réglage des paramètre pour la gridsearch
alphaslasso = np.linspace(0.1, 1, 5)
param_gridLasso = {'lasso__alpha': alphaslasso}

GridLasso, \
BestParametresLasso, \
ScoresLasso, \
TotalGHGEmissions_predLasso, \
figLasso = reg_modelGrid(model=Lasso(),
                            scaler=RobustScaler(quantile_range=(10, 90)),
                            X_train=BEBM_train,
                            X_test=BEBM_test,
                            y_train=TotalGHGEmissions_train,
                            y_test=TotalGHGEmissions_test,
                            y_test_name='TotalGHGEmissions_test',
                            y_pred_name='TotalGHGEmissions_predLasso',
                            score=score,
                            param_grid=param_gridLasso)

print(BestParametresLasso)
print(ScoresLasso)
figLasso.show()


      paramètre  Lasso()
0  lasso__alpha      1.0
         Lasso()
R²     -0.255703
RMSE  571.524475
MAE   123.408867


In [9]:
# graph visualisation RMSE Lasso pour les paramètres de GridSearchCV
FigRMSEGRidLasso = visuRMSEGrid(Lasso(), 'Lasso', alphaslasso, 'alpha',
                                GridLasso, None, None)
FigRMSEGRidLasso.show()
if write_data is True:
    FigRMSEGRidLasso.write_image('./Figures/EmissionsGraphRMSELasso.pdf')


 ### 1.1.4 Modèle ElasticNet

In [10]:
# régression elasticnet
# réglage des paramètre pour la gridsearch
alphasEN = np.logspace(-3, 3, 200)
l1ratioEN = np.linspace(0, 1, 6)
param_gridEN = {
    'elasticnet__alpha': alphasEN,
    'elasticnet__l1_ratio': l1ratioEN
}

GridEN, \
BestParametresEN, \
ScoresEN, \
TotalGHGEmissions_predEN, \
figEN = reg_modelGrid(model=ElasticNet(),
                         scaler=scaler,
                         X_train=BEBM_train,
                         X_test=BEBM_test,
                         y_train=TotalGHGEmissions_train,
                         y_test=TotalGHGEmissions_test,
                         y_test_name='TotalGHGEmissions_test',
                         y_pred_name='TotalGHGEmissions_predEN',
                         score=score,
                         param_grid=param_gridEN)

print(BestParametresEN)
print(ScoresEN)
figEN.show()


/home/lancelot/Documents/FormationDataScientist/P4/.env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.020e+08, tolerance: 9.395e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/lancelot/Documents/FormationDataScientist/P4/.env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.029e+08, tolerance: 8.776e+04 Linear regression models with null weight for the l1 regularization term

              paramètre  ElasticNet()
0     elasticnet__alpha     19.116441
1  elasticnet__l1_ratio      1.000000
      ElasticNet()
R²        0.183257
RMSE    460.929134
MAE     108.819280


In [11]:
# graph visualisation RMSE ElasticNet pour tout le meilleur paramètre l1 ratio
FigRMSEGRidEN = visuRMSEGrid(ElasticNet(), 'EN', alphasEN, 'alpha', GridEN,
                             BestParametresEN, 'elasticnet__l1_ratio')
FigRMSEGRidEN.show()
if write_data is True:
    FigRMSEGRidEN.write_image('./Figures/EmissionsGraphRMSEEN.pdf')


 ### 1.1.5 Modèle kNeighborsRegressor

In [12]:
# modèle kNN
# réglage des paramètre pour la gridsearch
n_neighbors = np.linspace(1, 100, dtype=int)
param_gridkNN = {'kneighborsregressor__n_neighbors': n_neighbors}


GridkNN, \
BestParametreskNN, \
ScoreskNN, \
TotalGHGEmissions_predkNN, \
figkNN = reg_modelGrid(model=KNeighborsRegressor(),
                         scaler=scaler,
                         X_train=BEBM_train,
                         X_test=BEBM_test,
                         y_train=TotalGHGEmissions_train,
                         y_test=TotalGHGEmissions_test,
                         y_test_name='TotalGHGEmissions_test',
                         y_pred_name='TotalGHGEmissions_predkNN',
                         score=score,
                         param_grid=param_gridkNN)

print(BestParametreskNN)
print(ScoreskNN)
figkNN.show()


                          paramètre  KNeighborsRegressor()
0  kneighborsregressor__n_neighbors                      5
      KNeighborsRegressor()
R²                 0.602850
RMSE             321.416814
MAE               91.582383


In [13]:
# graph visualisation RMSE kNN pour tout les paramètres de GridSearchCV
FigRMSEGRidkNN = visuRMSEGrid(KNeighborsRegressor(), 'kNN', n_neighbors,
                              'n neighbors', GridkNN)
FigRMSEGRidkNN.show()
if write_data is True:
    FigRMSEGRidkNN.write_image('./Figures/EmissionsGraphRMSEkNN.pdf')


 ### 1.1.6 Modèle RandomForestRegressor

In [14]:
# modèle RandomForestRegressor
# réglage des paramètre pour la gridsearch
n_estimatorsRF = np.logspace(0, 3, 10, dtype=int)
param_gridRF = {
    'randomforestregressor__n_estimators': n_estimatorsRF,
    'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'],
}

GridRF, \
BestParametresRF, \
ScoresRF, \
TotalGHGEmissions_predRF, \
figRF = reg_modelGrid(model=RandomForestRegressor(),
                         scaler=scaler,
                         X_train=BEBM_train,
                         X_test=BEBM_test,
                         y_train=TotalGHGEmissions_train.ravel(),
                         y_test=TotalGHGEmissions_test,
                         y_test_name='TotalGHGEmissions_test',
                         y_pred_name='TotalGHGEmissions_predRF',
                         score=score,
                         param_grid=param_gridRF)

print(BestParametresRF)
print(ScoresRF)
figRF.show()


                             paramètre RandomForestRegressor()
0  randomforestregressor__n_estimators                      46
1  randomforestregressor__max_features                    sqrt
      RandomForestRegressor()
R²                   0.719674
RMSE               270.036660
MAE                 72.363731


In [15]:
# graph visualisation RMSE RandomForestRegressor
# pour le meilleur paramètre max features
FigRMSEGRidRF = visuRMSEGrid(RandomForestRegressor(), 'RF', n_estimatorsRF,
                             'n estimators', GridRF, BestParametresRF,
                             'randomforestregressor__max_features')
FigRMSEGRidRF.show()
if write_data is True:
    FigRMSEGRidRF.write_image('./Figures/EmissionsGraphRMSERF.pdf')


 ### 1.1.7 Modèle AdaboostRegressor

In [16]:
# modèle AdaBoostRegressor
# réglage des paramètre pour la gridsearch
n_estimatorsAB = np.logspace(0, 2, 30, dtype=int)
param_gridAB = {
    'adaboostregressor__n_estimators': n_estimatorsAB,
    'adaboostregressor__loss': ['linear', 'square', 'exponential']
}

GridAB, \
BestParametresAB, \
ScoresAB, \
TotalGHGEmissions_predAB, \
figAB = reg_modelGrid(model=AdaBoostRegressor(),
                         scaler=scaler,
                         X_train=BEBM_train,
                         X_test=BEBM_test,
                         y_train=TotalGHGEmissions_train.ravel(),
                         y_test=TotalGHGEmissions_test,
                         y_test_name='TotalGHGEmissions_test',
                         y_pred_name='TotalGHGEmissions_predAB',
                         score=score,
                         param_grid=param_gridAB)

print(BestParametresAB)
print(ScoresAB)
figAB.show()


                         paramètre AdaBoostRegressor()
0  adaboostregressor__n_estimators                  20
1          adaboostregressor__loss              square
      AdaBoostRegressor()
R²               0.650471
RMSE           301.531675
MAE            102.054896


In [17]:
# graph visualisation RMSE AdaBoostRegressor
# pour le meilleur paramètre loss
FigRMSEGRidAB = visuRMSEGrid(AdaBoostRegressor(), 'AB', n_estimatorsAB,
                             'n estimators', GridAB, BestParametresAB,
                             'adaboostregressor__loss')
FigRMSEGRidAB.show()
if write_data is True:
    FigRMSEGRidAB.write_image('./Figures/EmissionsGraphRMSEAB.pdf')


## 1.2 Émissions au log

In [18]:
TotalGHGEmissions_train_log = np.log2(1 + TotalGHGEmissions_train)
TotalGHGEmissions_test_log = np.log2(1 + TotalGHGEmissions_test)


 ### 1.2.1 Modèle LinearRegression

In [35]:
# modèle régression linéaire
pipeLR = make_pipeline(scaler, LinearRegression())

pipeLR.fit(BEBM_train, TotalGHGEmissions_train_log)

TotalGHGEmissions_pred_logLR = pipeLR.predict(BEBM_test)

LRr2_log = metrics.r2_score(TotalGHGEmissions_test_log,
                            TotalGHGEmissions_pred_logLR)
print("r2 :", LRr2)
LRrmse_log = metrics.mean_squared_error(TotalGHGEmissions_test_log,
                                        TotalGHGEmissions_pred_logLR,
                                        squared=False)
print("rmse :", LRrmse)

fig = px.scatter(
    x=TotalGHGEmissions_pred_logLR.squeeze(),
    y=TotalGHGEmissions_test_log.squeeze(),
    labels={
        'x': f'{TotalGHGEmissions_pred_logLR=}'.partition('=')[0],
        'y': f'{TotalGHGEmissions_test_log=}'.partition('=')[0]
    },
    title=
    "Visualisation des données d'émissions prédites par le modèle de régression linéaire<br>vs les données test"
)
fig.show()

r2 : -0.3095599261157267
rmse : 583.6520545460625


 ### 1.2.2 Modèle Ridge

In [20]:
# régression ridge
# réglage des paramètre pour la gridsearch
alphasridge_log = np.logspace(-3, 5, 1000)
param_gridRidge_log = {'ridge__alpha': alphasridge_log}

GridRidge_log, \
BestParametresRidge_log, \
ScoresRidge_log, \
TotalGHGEmissions_pred_logRidge_log, \
figRidge_log = reg_modelGrid(model=Ridge(),
                            scaler=scaler,
                            X_train=BEBM_train,
                            X_test=BEBM_test,
                            y_train=TotalGHGEmissions_train_log,
                            y_test=TotalGHGEmissions_test_log,
                            y_test_name='TotalGHGEmissions_test_log',
                            y_pred_name='TotalGHGEmissions_pred_logRidge',
                            score=score,
                            param_grid=param_gridRidge_log)

print(BestParametresRidge_log)
print(ScoresRidge_log)
figRidge_log.show()


      paramètre     Ridge()
0  ridge__alpha  572.476624
       Ridge()
R²   -0.425062
RMSE  2.512048
MAE   1.550505


In [21]:
# graph visualisation RMSE Ridge pour tout les paramètres de GridSearchCV
FigRMSEGRidRidge_log = visuRMSEGrid(Ridge(), 'Ridge', alphasridge_log, 'alpha',
                                    GridRidge_log)
FigRMSEGRidRidge_log.show()
if write_data is True:
    FigRMSEGRidRidge_log.write_image(
        './Figures/EmissionsGraphRMSERidge_log.pdf')

 ### 1.2.3 Modèle Lasso

In [22]:
# régression lasso
# réglage des paramètre pour la gridsearch
alphaslasso_log = np.linspace(0.1, 1, 5)
param_gridLasso_log = {'lasso__alpha': alphaslasso_log}

GridLasso_log, \
BestParametresLasso_log, \
ScoresLasso_log, \
TotalGHGEmissions_pred_logLasso_log, \
figLasso_log = reg_modelGrid(model=Lasso(),
                            scaler=RobustScaler(quantile_range=(10, 90)),
                            X_train=BEBM_train,
                            X_test=BEBM_test,
                            y_train=TotalGHGEmissions_train_log,
                            y_test=TotalGHGEmissions_test_log,
                            y_test_name='TotalGHGEmissions_test_log',
                            y_pred_name='TotalGHGEmissions_pred_logLasso',
                            score=score,
                            param_grid=param_gridLasso_log)

print(BestParametresLasso_log)
print(ScoresLasso_log)
figLasso_log.show()


      paramètre  Lasso()
0  lasso__alpha      0.1
       Lasso()
R²   -0.370634
RMSE  2.463609
MAE   1.552973


In [23]:
# graph visualisation RMSE Lasso pour tout les paramètres de GridSearchCV
FigRMSEGRidLasso_log = visuRMSEGrid(Lasso(), 'Lasso', alphaslasso_log, 'alpha',
                                    GridLasso_log, None, None)
FigRMSEGRidLasso_log.show()
if write_data is True:
    FigRMSEGRidLasso_log.write_image(
        './Figures/EmissionsGraphRMSELasso_log.pdf')


 ### 1.2.4 Modèle ElasticNet

In [24]:
# régression elasticnet
# réglage des paramètre pour la gridsearch
alphasEN_log = np.logspace(-3, 1, 30)
l1ratioEN_log = np.linspace(0, 1, 6)
param_gridEN_log = {
    'elasticnet__alpha': alphasEN_log,
    'elasticnet__l1_ratio': l1ratioEN_log
}

GridEN_log, \
BestParametresEN_log, \
ScoresEN_log, \
TotalGHGEmissions_pred_logEN, \
figEN_log = reg_modelGrid(model=ElasticNet(),
                         scaler=scaler,
                         X_train=BEBM_train,
                         X_test=BEBM_test,
                         y_train=TotalGHGEmissions_train_log,
                         y_test=TotalGHGEmissions_test_log,
                         y_test_name='TotalGHGEmissions_test_log',
                         y_pred_name='TotalGHGEmissions_pred_logEN',
                         score=score,
                         param_grid=param_gridEN_log)

print(BestParametresEN_log)
print(ScoresEN_log)
figEN_log.show()


/home/lancelot/Documents/FormationDataScientist/P4/.env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.533e+03, tolerance: 1.777e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/home/lancelot/Documents/FormationDataScientist/P4/.env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.442e+03, tolerance: 1.756e+00 Linear regression models with null weight for the l1 regularization term

              paramètre  ElasticNet()
0     elasticnet__alpha      0.062102
1  elasticnet__l1_ratio      1.000000
      ElasticNet()
R²       -0.435249
RMSE      2.521010
MAE       1.551599


In [25]:
# graph visualisation RMSE ElasticNet pour tout le meilleur paramètre l1 ratio
FigRMSEGRidEN_log = visuRMSEGrid(ElasticNet(), 'EN', alphasEN_log, 'alpha',
                                 GridEN_log, BestParametresEN_log,
                                 'elasticnet__l1_ratio')
FigRMSEGRidEN_log.show()
if write_data is True:
    FigRMSEGRidEN_log.write_image('./Figures/EmissionsGraphRMSEEN_log.pdf')


 ### 1.2.5 Modèle kNeighborsRegressor

In [26]:
# modèle kNN
# réglage des paramètre pour la gridsearch
n_neighbors_log = np.linspace(1, 100, dtype=int)
param_gridkNN_log = {'kneighborsregressor__n_neighbors': n_neighbors_log}


GridkNN_log, \
BestParametreskNN_log, \
ScoreskNN_log, \
TotalGHGEmissions_pred_logkNN_log, \
figkNN_log = reg_modelGrid(model=KNeighborsRegressor(),
                         scaler=scaler,
                         X_train=BEBM_train,
                         X_test=BEBM_test,
                         y_train=TotalGHGEmissions_train_log,
                         y_test=TotalGHGEmissions_test_log,
                         y_test_name='TotalGHGEmissions_test_log',
                         y_pred_name='TotalGHGEmissions_pred_logkNN',
                         score=score,
                         param_grid=param_gridkNN_log)

print(BestParametreskNN_log)
print(ScoreskNN_log)
figkNN_log.show()


                          paramètre  KNeighborsRegressor()
0  kneighborsregressor__n_neighbors                     45
      KNeighborsRegressor()
R²                 0.432091
RMSE               1.585807
MAE                1.284268


In [27]:
# graph visualisation RMSE kNN pour les paramètres de GridSearchCV
FigRMSEGRidkNN_log = visuRMSEGrid(KNeighborsRegressor(), 'kNN',
                                  n_neighbors_log, 'n neighbors', GridkNN_log)
FigRMSEGRidkNN_log.show()
if write_data is True:
    FigRMSEGRidkNN_log.write_image('./Figures/EmissionsGraphRMSEkNN_log.pdf')


 ### 1.2.5 Modèle RandomForestRegressor

In [28]:
# modèle RandomForestRegressor
# réglage des paramètre pour la gridsearch
n_estimatorsRF_log = np.logspace(0, 3, 10, dtype=int)
param_gridRF_log = {
    'randomforestregressor__n_estimators': n_estimatorsRF_log,
    'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'],
}

GridRF_log, \
BestParametresRF_log, \
ScoresRF_log, \
TotalGHGEmissions_pred_logRF_log, \
figRF_log = reg_modelGrid(model=RandomForestRegressor(),
                         scaler=scaler,
                         X_train=BEBM_train,
                         X_test=BEBM_test,
                         y_train=TotalGHGEmissions_train_log.ravel(),
                         y_test=TotalGHGEmissions_test_log,
                         y_test_name='TotalGHGEmissions_test_log',
                         y_pred_name='TotalGHGEmissions_pred_log_logRF',
                         score=score,
                         param_grid=param_gridRF)

print(BestParametresRF_log)
print(ScoresRF_log)
figRF_log.show()


                             paramètre RandomForestRegressor()
0  randomforestregressor__n_estimators                     464
1  randomforestregressor__max_features                    auto
      RandomForestRegressor()
R²                   0.685788
RMSE                 1.179566
MAE                  0.861599


In [29]:
# graph visualisation RMSE RandomForestRegressor
# pour le meilleur paramètre max features
FigRMSEGRidRF_log = visuRMSEGrid(RandomForestRegressor(), 'RF',
                                 n_estimatorsRF_log, 'n estimators',
                                 GridRF_log, BestParametresRF_log,
                                 'randomforestregressor__max_features')
FigRMSEGRidRF_log.show()
if write_data is True:
    FigRMSEGRidRF_log.write_image('./Figures/EmissionsGraphRMSERF_log.pdf')


 ### 1.2.7 Modèle AdaboostRegressor

In [30]:
# modèle AdaBoostRegressor
# réglage des paramètre pour la gridsearch
n_estimatorsAB_log = np.logspace(0, 2, 30, dtype=int)
param_gridAB_log = {
    'adaboostregressor__n_estimators': n_estimatorsAB_log,
    'adaboostregressor__loss': ['linear', 'square', 'exponential']
}

GridAB_log, \
BestParametresAB_log, \
ScoresAB_log, \
TotalGHGEmissions_pred_logAB, \
figAB_log = reg_modelGrid(model=AdaBoostRegressor(),
                         scaler=scaler,
                         X_train=BEBM_train,
                         X_test=BEBM_test,
                         y_train=TotalGHGEmissions_train_log.ravel(),
                         y_test=TotalGHGEmissions_test_log,
                         y_test_name='TotalGHGEmissions_test_log',
                         y_pred_name='TotalGHGEmissions_predAB_log',
                         score=score,
                         param_grid=param_gridAB_log)

print(BestParametresAB_log)
print(ScoresAB_log)
figAB_log.show()


                         paramètre AdaBoostRegressor()
0  adaboostregressor__n_estimators                   4
1          adaboostregressor__loss              linear
      AdaBoostRegressor()
R²               0.417478
RMSE             1.606080
MAE              1.326605


In [31]:
# graph visualisation RMSE AdaBoostRegressor
# pour le meilleur paramètre loss
FigRMSEGRidAB_log = visuRMSEGrid(AdaBoostRegressor(), 'AB', n_estimatorsAB_log,
                             'n estimators', GridAB_log, BestParametresAB_log,
                             'adaboostregressor__loss')
FigRMSEGRidAB_log.show()
if write_data is True:
    FigRMSEGRidAB_log.write_image('./Figures/EmissionsGraphRMSEAB_log.pdf')


In [32]:
Scores = ScoresLasso.join(
    [ScoresRidge, ScoresEN, ScoreskNN, ScoresRF, ScoresAB])


In [33]:
ScoresLog = ScoresLasso_log.join(
    [ScoresRidge_log, ScoresEN_log, ScoreskNN_log, ScoresRF_log, ScoresAB_log])


In [34]:
CompareScores = Scores.join(ScoresLog, rsuffix='_log')
if write_data is True:
    CompareScores.to_latex('./Tableaux/EmmisionsScoresModèles.tex')
